In [1]:
from transformers import AutoModel, AutoTokenizer
AutoTokenizer.from_pretrained('almanach/camembert-base')
AutoModel.from_pretrained('almanach/camembert-base')

c:\Users\bausm\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CamembertModel(
  (embeddings): CamembertEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): CamembertEncoder(
    (layer): ModuleList(
      (0-11): 12 x CamembertLayer(
        (attention): CamembertAttention(
          (self): CamembertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): CamembertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
 

In [11]:
tokenize = AutoTokenizer.from_pretrained("almanach/camembert-base")


tok_sent = tokenize(["Arturo","devra","retenter","demain"], is_split_into_words=True, return_tensors='pt')
print(tok_sent['input_ids'])


model=AutoModel.from_pretrained("almanach/camembert-base")
emb_sent = model(**tok_sent)['last_hidden_state'][0]
print(emb_sent.shape)

tok_sent.word_ids()
print(tokenize.convert_ids_to_tokens(tok_sent['input_ids'][0]))


tensor([[    5,  2082, 10394,  2613,   343,  3889,   108,  2385,     6]])
torch.Size([9, 768])
['<s>', '▁Art', 'uro', '▁devra', '▁re', 'tent', 'er', '▁demain', '</s>']


In [44]:
# Extraction de sequoia
import sys
sys.path.append("../TP1/")
from conllu import parse_incr
import numpy as np

phrases = []
for sent in parse_incr(open("../TP1/sequoia-ud.parseme.frsemcor.simple.small", encoding='UTF-8')):
    mots = [tok["form"] for tok in sent if tok["upos"] in ["NOUN", "PROPN", "NUM"]]
    if mots:
        phrases.append(mots)

print(phrases[:10])


tokenize = AutoTokenizer.from_pretrained("almanach/camembert-base")

embeddings = []
for phrase in phrases:
    tok_sents = tokenize(phrase, is_split_into_words=True, return_tensors='pt', padding=True, truncation=True)
    model = AutoModel.from_pretrained("almanach/camembert-base")
    emb_sent = model(**tok_sents)['last_hidden_state'][0]
    embeddings.append(emb_sent)

tok_sents.word_ids() 
word_embeddings = []
for i, word_id in enumerate(tok_sents.word_ids()):
    if word_id is not None and phrase[word_id] in mots:
        word_embeddings.append(emb_sent[i].detach().numpy())

# Moyennage des embeddings des sub-tokens pour chaque mot
final_embeddings = []
for word in mots:
    word_embs = [emb for i, emb in enumerate(word_embeddings) if tok_sents.word_ids()[i] == mots.index(word)]
    if word_embs:
        final_embeddings.append(np.mean(word_embs, axis=0))

print(final_embeddings)


[['association', 'décors', 'aide', 'bénévoles', 'tableaux', 'thème', 'Cinq', 'siècles', 'activité', 'région', 'Ancerville'], ['Ouverture', 'jours', 'lundi', '14', 'h', '30', '18', 'h'], ['sous-préfet', 'énergie', 'réalisation'], ['membres', 'club', 'occasion', 'après-midi', '14', 'h', 'salle', 'Jean-Mathieu'], ['M.', 'Hosneld', '44', 'ans'], ['cas', 'Joseph', 'Bari', 'comité', 'vedette', 'colombophilie', '1962', 'époque', 'pigeonnier', 'Châtenois-les-Forges', 'espace', 'Grand-Charmont'], ['soirée', 'François', 'Puel', 'chercheur', 'laboratoire', 'astrophysique', 'Besançon'], ['Princes', 'nombre', 'roi', 'Pierre', 'public', 'souvenirs', 'enfance', 'temps', 'représentation'], ['événement', 'deux', 'établissements', 'mercredi', '23', 'juin', 'occasion', 'assemblée', 'Offices', 'tourisme', 'Syndicats', 'initiative', 'Doubs', 'jour', 'présidence', 'Edmond', 'Maire', 'Saline'], ['Suzanne', 'Collin', 'Brauvilliers', 'Meuse', '21', 'janvier', '1924', 'famille', 'cinq', 'enfants']]
[array([ 5.2

In [52]:
# Extraction de sequoia
import sys
sys.path.append("../TP1/")
from conllu import parse_incr
import numpy as np
import torch

phrases = []
for sent in parse_incr(open("../TP1/sequoia-ud.parseme.frsemcor.simple.small", encoding='UTF-8')):
    mots = [tok["form"] for tok in sent if tok["upos"] in ["NOUN", "PROPN", "NUM"]]
    if mots:
        phrases.append(mots)

print(phrases[:10])

# Assuming super_sense_labels is a list of super-sense labels corresponding to each word in 'mots'
super_sense_labels = ["label1", "label2", "label3", "label4", "label5", "label6", "label7", "label8", "label9", "label10", "label11", "label12", "label13", "label14"]

# Convert super-sense labels to indices
label_to_index = {label: idx for idx, label in enumerate(set(super_sense_labels))}
super_sense_indices = [label_to_index[label] for label in super_sense_labels]

print(super_sense_indices)

# Tokenize and embed phrases
final_embeddings = []
embeddings = []
tokenize = AutoTokenizer.from_pretrained("almanach/camembert-base")
model = AutoModel.from_pretrained("almanach/camembert-base")

for phrase in phrases:
    tok_sents = tokenize(phrase, is_split_into_words=True, return_tensors='pt', padding=True, truncation=True)
    emb_sent = model(**tok_sents)['last_hidden_state'][0]
    embeddings.append(emb_sent)

    word_embeddings = []
    for i, word_id in enumerate(tok_sents.word_ids()):
        if word_id is not None and phrase[word_id] in mots:
            word_embeddings.append(emb_sent[i].detach().numpy())

    # Moyennage des embeddings des sub-tokens pour chaque mot
    for word in phrase:
        word_embs = [emb for i, emb in enumerate(word_embeddings) if tok_sents.word_ids()[i] == phrase.index(word)]
        if word_embs:
            final_embeddings.append(np.mean(word_embs, axis=0))

print(final_embeddings)
print(np.asarray(final_embeddings).shape)

# Freeze embeddings with torch.no_grad()
with torch.no_grad():
    for phrase in phrases:
        tok_sents = tokenize(phrase, is_split_into_words=True, return_tensors='pt', padding=True, truncation=True)
        emb_sent = model(**tok_sents)['last_hidden_state'][0]
        embeddings.append(emb_sent)

[['association', 'décors', 'aide', 'bénévoles', 'tableaux', 'thème', 'Cinq', 'siècles', 'activité', 'région', 'Ancerville'], ['Ouverture', 'jours', 'lundi', '14', 'h', '30', '18', 'h'], ['sous-préfet', 'énergie', 'réalisation'], ['membres', 'club', 'occasion', 'après-midi', '14', 'h', 'salle', 'Jean-Mathieu'], ['M.', 'Hosneld', '44', 'ans'], ['cas', 'Joseph', 'Bari', 'comité', 'vedette', 'colombophilie', '1962', 'époque', 'pigeonnier', 'Châtenois-les-Forges', 'espace', 'Grand-Charmont'], ['soirée', 'François', 'Puel', 'chercheur', 'laboratoire', 'astrophysique', 'Besançon'], ['Princes', 'nombre', 'roi', 'Pierre', 'public', 'souvenirs', 'enfance', 'temps', 'représentation'], ['événement', 'deux', 'établissements', 'mercredi', '23', 'juin', 'occasion', 'assemblée', 'Offices', 'tourisme', 'Syndicats', 'initiative', 'Doubs', 'jour', 'présidence', 'Edmond', 'Maire', 'Saline'], ['Suzanne', 'Collin', 'Brauvilliers', 'Meuse', '21', 'janvier', '1924', 'famille', 'cinq', 'enfants']]
[1, 10, 4, 3